In [441]:
import matplotlib.pyplot as plt
import numpy as np
import random
from collections import defaultdict
from sklearn import preprocessing
from scipy import stats
import sys
from math import ceil, sqrt

%matplotlib inline

TRAINING_DATA = "census_data/train_data.csv"
TEST_DATA = "census_data/test_data.csv"
UNKNOWN_VALUE_CONST = -1

categorical_variable_indexes = [1, 3, 5, 6, 7, 8, 9, 13]

In [522]:
# 1 = income >= 50k, 0 = income < 50k
def load_data():
    # process training data
    f = open(TRAINING_DATA, "r")
    features = np.array(f.readline()[:-1].split(',')[:-1])
    data = []
    for (i, line) in enumerate(f):
        line = line[:-1]
        data.append(np.array(line.split(',')))
    data = np.array(data)
    training_data, training_labels = data[:,:-1], data[:,-1].astype(int)
    f.close()
    
    # process test data
    f = open(TEST_DATA, "r")
    f.readline()
    test_data = []
    for i,line in enumerate(f):
        line = line[:-1]
        test_data.append(np.array(line.split(',')))
    test_data = np.array(test_data)
    return features, training_data, training_labels, test_data

features, pre_training_data, training_labels, pre_test_data = load_data()

In [525]:
def preprocess_data(pre_training_data, pre_test_data):
    training_data, test_data = pre_training_data, pre_test_data
    for category_index in categorical_variable_indexes:
        le = preprocessing.LabelEncoder()
        le.fit(np.concatenate((training_data[:,category_index], test_data[:,category_index])))
        training_data[:,category_index] = le.transform(training_data[:,category_index])
        test_data[:,category_index] = le.transform(test_data[:,category_index])
        # replace unknown value (?) with "-1"
        if "?" in le.classes_:
            unknown_index = le.transform(["?"])[0]
            column = training_data[:,category_index].astype(int)
            training_data[:,category_index][column == unknown_index] = UNKNOWN_VALUE_CONST
            column = test_data[:,category_index].astype(int)
            test_data[:,category_index][column == unknown_index] = UNKNOWN_VALUE_CONST
        
    return training_data, test_data

training_data, test_data = preprocess_data(pre_training_data, pre_test_data)
training_data = training_data.astype(int)
test_data = test_data.astype(int)

In [444]:
def entropy(data):
    if data is None or len(data) == 0:
        return 0.0
    counter = defaultdict(lambda: 0)
    for item in data:
        counter[item] += 1
    en, num_items = 0.0, len(data)
    for key, value in counter.items():
        p = value / num_items
        en = en - p * np.log2(p)
    return en

assert entropy([1, 1, 2, 3]) == 1.5
assert abs(entropy([100, 100, 125, 150, 150]) - 1.5219280948873622) < 1e-5

In [445]:
def two_way_splits(length):
    splits_list = []
    if length <= 21:
        for i in range(1, 1 << (length - 1)):
            current, other = [], []
            for j in range(length):
                if (i >> j) & 1 == 1:
                    current.append(j)
                else:
                    other.append(j)
            splits_list.append([current, other])
        if len(splits_list) > MAX_NUM_FEATURES:
            splits_list = np.array(splits_list)
            random_indexes = np.random.choice(len(splits_list), MAX_NUM_FEATURES, replace=False)
            splits_list = splits_list[random_indexes]
    else:
        for iteration in range(MAX_NUM_FEATURES):
            num_left_subset = random.randint(1, length - 1)
            num_right_subset = length - num_left_subset
            left_indexes = np.random.choice(length, num_left_subset, replace=False)
            right_indexes = np.delete(np.arange(length), left_indexes)
            splits_list.append([left_indexes, right_indexes])
    return splits_list

In [446]:
class Node:
    def __init__(self, left, right, split_feature, is_leaf_node):
        self.left = left
        self.right = right
        self.split_feature = split_feature
        self.is_leaf_node = is_leaf_node
        
class QuantitativeNode(Node):
    def __init__(self, left, right, split_feature, split_threshold):
        super(QuantitativeNode, self).__init__(left, right, split_feature, False)
        self.is_categorical_node = False
        self.split_threshold = split_threshold
    
class CategoricalNode(Node):
    def __init__(self, left, right, split_feature, left_subset_vals, right_subset_vals):
        super(CategoricalNode, self).__init__(left, right, split_feature, False)
        self.is_categorical_node = True
        # values that go into left, right nodes
        self.left_subset_vals = left_subset_vals
        self.right_subset_vals = right_subset_vals

class LeafNode(Node):
    def __init__(self, label):
        super(LeafNode, self).__init__(None, None, None, True)
        self.label = label

In [516]:
class DecisionTree:
    def __init__(self, category_vars, attr_bagging=False, use_limit_heuristics=False):
        self.category_vars = category_vars
        self.attr_bagging = attr_bagging
        self.use_limit_heuristics = use_limit_heuristics
        
    def build_tree(self, data_indexes, training_data, training_labels, current_height):
        assert data_indexes.shape[0] > 0
        
        cur_training_data = training_data[data_indexes]
        cur_training_labels = training_labels[data_indexes]
        mode = stats.mode(cur_training_labels, axis=None)
        mode_percentage = mode[1][0] / cur_training_labels.shape[0]
        usable_features = np.array([index for index in range(self.M) if np.unique(cur_training_data[:,index]).shape[0] > 1])

        if np.unique(cur_training_labels).shape[0] == 1 or \
            usable_features.shape[0] == 0 or \
            (self.use_limit_heuristics and (current_height == HEIGHT_STOP_THRESHOLD or \
            mode_percentage >= MODE_LABEL_STOP_THRESHOLD or data_indexes.shape[0] <= VOLUME_STOP_THRESHOLD)):
            return LeafNode(mode[0][0])
        
        if self.attr_bagging:
            feature_indexes = np.random.choice(len(usable_features), ceil(sqrt(len(usable_features))), replace=False)
        else:
            feature_indexes = np.arange(0, len(usable_features))
        feature_indexes = usable_features[np.array(feature_indexes, dtype=np.int)]
        split_data = self.segmenter(cur_training_data, cur_training_labels, feature_indexes, \
                                    data_indexes, self.information_gain, np.mean)
        
        is_categorical_feature, best_feature = split_data[0], split_data[1]
        if is_categorical_feature:
            left_subset_vals, right_subset_vals = split_data[2]
        else:
            split_threshold = split_data[2]
            
        best_left_indexes, best_right_indexes = split_data[3], split_data[4]
        
        left_node = self.build_tree(best_left_indexes, training_data, training_labels, current_height + 1)
        right_node = self.build_tree(best_right_indexes, training_data, training_labels, current_height + 1)
        if is_categorical_feature:
            current_node = CategoricalNode(left_node, right_node, best_feature, \
                                           left_subset_vals, right_subset_vals)
        else:
            current_node = QuantitativeNode(left_node, right_node, best_feature, split_threshold)
        return current_node
    
    def train(self, training_data, training_labels):        
        self.N = training_data.shape[0]
        self.M = training_data.shape[1]
        
        data_indexes = np.arange(0, training_data.shape[0])
        self.root = self.build_tree(data_indexes, training_data, training_labels, 1)
        
    def predict_recursive(self, current_node, data_item):
        if current_node.is_leaf_node:
            return current_node.label

        feature_value = data_item[current_node.split_feature]
        if current_node.is_categorical_node:
            if feature_value in current_node.left_subset_vals:
                return self.predict_recursive(current_node.left, data_item)
            else:
                return self.predict_recursive(current_node.right, data_item)
        else:
            if feature_value <= current_node.split_threshold:
                return self.predict_recursive(current_node.left, data_item)
            else:
                return self.predict_recursive(current_node.right, data_item)
    
    def predict(self, test_data):
        predictions = []
        for data_item in test_data:
            predictions.append(self.predict_recursive(self.root, data_item))
        return np.array(predictions)
    
    def information_gain(self, left_data, right_data):
        
        parent_data = np.hstack((left_data, right_data))
        left_len = len(left_data)
        right_len = len(right_data)
        total_len = left_len + right_len
        return entropy(parent_data) - \
               (left_len / total_len) * entropy(left_data) - \
               (right_len / total_len) * entropy(right_data)
    
    def impurity(self, left_label_hist, right_label_hist):
        """
        A method that takes in the result of a split: two histograms
        (a histogram is a mapping from label values to their frequencies)
        that count the frequencies of labels on the ”left” and ”right”
        side of that split. The method calculates and outputs a scalar
        value representing the impurity (i.e. the ”badness”) of the specified
        split on the input data.
        """
        total_count = sum(left_label_hist.values())
        val = 0.0
        for key, value in left_label_hist.items():
            val += (value * value / total_count)
        for key, value in right_label_hist.items():
            val += (value * value / total_count)    
        return -val
    
    def segmenter(self, data, labels, features_used, indexes, metric_func, unknown_predictor_func):
        """
        A method that takes in data and labels. When called, it finds the 
        best split rule for a Node using the impurity measure and input data.
        There are many different types of segmenters you might implement,
        each with a different method of choosing a threshold. The usual method
        is exhaustively trying lots of different threshold values from the data
        and choosing the combination of split feature and threshold with the
        lowest impurity value. The final split rule uses the split feature with
        the lowest impurity value and the threshold chosen by the segmenter.
        """
        max_metric, best_feature, best_left_indexes, best_right_indexes = \
            -1e100, None, None, None
        left_subset_vals, right_subset_vals, split_threshold = None, None, None

        for feature_number in features_used:
            feature_list = data[:,feature_number]
            feature_list[feature_list == UNKNOWN_VALUE_CONST] = unknown_predictor_func(feature_list)
            
            unique_sorted_features = np.sort(np.unique(feature_list))
            if unique_sorted_features.shape[0] >= MAX_NUM_FEATURES:
                unique_sorted_features = unique_sorted_features[::unique_sorted_features.shape[0]//MAX_NUM_FEATURES]
            
            feature_list_with_indexes = np.dstack((feature_list, np.arange(feature_list.shape[0])))[0]
            feature_list_with_indexes = feature_list_with_indexes[feature_list_with_indexes[:,0].argsort()]

            if feature_number in self.category_vars:
                # categorical variables
                subset_splits = two_way_splits(unique_sorted_features.shape[0])

                for (i, (left_idxs, right_idxs)) in enumerate(subset_splits):
                    left_vals = unique_sorted_features[left_idxs]
                    right_vals = unique_sorted_features[right_idxs]
                    
                    left_indexes = np.where(np.in1d(feature_list, left_vals))[0]
                    right_indexes = np.where(np.in1d(feature_list, right_vals))[0]
                    info_gain = metric_func(labels[left_indexes], labels[right_indexes])
                    
                    if info_gain > max_metric or \
                        (info_gain == max_metric and \
                         abs(len(left_indexes) - len(right_indexes)) < abs(len(best_left_indexes) - len(best_right_indexes.shape))):
                        left_subset_vals = left_vals
                        right_subset_vals = right_vals
                        
                        max_metric = info_gain
                        best_left_indexes, best_right_indexes = indexes[left_indexes], indexes[right_indexes]
                        best_feature = feature_number
            else:
                # quantitative variables
                left_indexes, right_indexes = [], feature_list_with_indexes[:,1][::-1].tolist()
                pointer_index = 0
                for split_value in unique_sorted_features[:-1]:
                    while feature_list_with_indexes[pointer_index][0] <= split_value:
                        left_indexes.append(feature_list_with_indexes[pointer_index][1])
                        right_indexes.pop()
                        pointer_index += 1
                    
                    left_indexes_np = np.array(left_indexes, dtype=np.int)
                    right_indexes_np = np.array(right_indexes, dtype=np.int)
                    info_gain = metric_func(labels[left_indexes_np], labels[right_indexes_np])                    
                    
                    if info_gain > max_metric or \
                        (info_gain == max_metric and \
                         abs(len(left_indexes) - len(right_indexes)) < abs(len(best_left_indexes) - len(best_right_indexes.shape))):
                        split_threshold = split_value
                        
                        max_metric = info_gain
                        best_left_indexes, best_right_indexes = indexes[left_indexes_np], indexes[right_indexes_np]
                        best_feature = feature_number
                        
        is_categorical_feature = best_feature in self.category_vars
        if is_categorical_feature:
            return (is_categorical_feature, best_feature, (left_subset_vals, right_subset_vals), \
                    best_left_indexes, best_right_indexes)
        else:
            return (is_categorical_feature, best_feature, split_threshold, \
                    best_left_indexes, best_right_indexes)
        
classifier = DecisionTree([0], use_limit_heuristics=False)
X = np.array([ \
    [1, 100, 2],
    [2, 150, 3],
    [2, 125, 2],
    [1, 125, 5],
    [1, 150, 4]
])
y = np.array([1, 0, 1, 0, 1])
classifier.train(X, y)
assert np.array_equal(classifier.predict(X), y)

is leaf node False
split feature, value fnlwgt 2
split threshold 2
is leaf node True
is leaf node False
split feature, value fnlwgt 3
split threshold 2
is leaf node False
split feature, value age 2
subset vals [1] [2]
is leaf node True
is leaf node False
split feature, value fnlwgt 2
split threshold 2
is leaf node True
is leaf node False
split feature, value fnlwgt 5
split threshold 2
is leaf node False
split feature, value age 1
subset vals [1] [2]
is leaf node False
split feature, value workclass 125
split threshold 125
is leaf node True
is leaf node False
split feature, value fnlwgt 4
split threshold 2
is leaf node False
split feature, value age 1
subset vals [1] [2]
is leaf node False
split feature, value workclass 150
split threshold 125
is leaf node True


In [ ]:
MODE_LABEL_STOP_THRESHOLD = 0.98
HEIGHT_STOP_THRESHOLD = 15
VOLUME_STOP_THRESHOLD = 10
MAX_NUM_FEATURES = 1024
tree_classifier = DecisionTree(categorical_variable_indexes, use_limit_heuristics=True)
tree_classifier.train(training_data, training_labels)
tree_prediction = tree_classifier.predict(training_data)

In [499]:
print(np.sum(prediction == training_labels) / training_data.shape[0])
tree_test_prediction = tree_classifier.predict(test_data)

0.839933993399


In [491]:
with open("decision_tree_census.csv", "w") as f:
    f.write("Id,Category\n")
    for i, val in enumerate(tree_test_prediction):
        f.write("{},{}\n".format(i+1, val))
    f.close()

In [493]:
class RandomForest(DecisionTree):
    def __init__(self, category_vars, num_trees, attr_bagging=True, use_limit_heuristics=True):
        super(RandomForest, self).__init__(category_vars, attr_bagging, use_limit_heuristics)
        self.num_trees = num_trees
        
    def generate_random_tree(self):
        # data bagging
        bag_indexes = np.random.choice(self.N, self.N, replace=True)
        training_data_bag, training_labels_bag = \
            self.training_data[bag_indexes], self.training_labels[bag_indexes]
        data_indexes = np.arange(0, self.N)

        return self.build_tree(data_indexes, training_data_bag, training_labels_bag, 1)
        
    def train(self, training_data, training_labels):
        self.N = training_data.shape[0]
        self.M = training_data.shape[1]
        self.training_data = training_data
        self.training_labels = training_labels
        
        data_indexes = np.arange(0, training_data.shape[0])
        
        self.roots = []
        
        for i in range(self.num_trees):
            if i % (self.num_trees // 10) == 0:
                print("Generating tree {} of {}..".format(i, self.num_trees))
                sys.stdout.flush()
            self.roots.append(self.generate_random_tree())
            
    def predict(self, test_data):
        predictions = []
        for data_item in test_data:
            forest_predictions = [self.predict_recursive(root, data_item) for root in self.roots]
            mode_value = stats.mode(forest_predictions)[0][0]
            predictions.append(mode_value)
        return np.array(predictions)
    
classifier = RandomForest([0], 50, use_limit_heuristics=False)
X = np.array([ \
    [1, 100, 2],
    [2, 150, 3],
    [1, 125, 1],
    [2, 125, 5],
    [1, 150, 4],
    [2, 100, 3]
])
y = np.array([1, 0, 1, 0, 1, 1])
classifier.train(X, y)
assert np.array_equal(classifier.predict(X), y)

Generating tree 0 of 50..
Generating tree 5 of 50..
Generating tree 10 of 50..
Generating tree 15 of 50..
Generating tree 20 of 50..
Generating tree 25 of 50..
Generating tree 30 of 50..
Generating tree 35 of 50..
Generating tree 40 of 50..
Generating tree 45 of 50..


In [495]:
MODE_LABEL_STOP_THRESHOLD = 0.90
HEIGHT_STOP_THRESHOLD = 5
VOLUME_STOP_THRESHOLD = 50
MAX_NUM_FEATURES = 128
random_forest_classifier = RandomForest(categorical_variable_indexes, 50, use_limit_heuristics=True)
random_forest_classifier.train(training_data, training_labels)
random_training_prediction = random_forest_classifier.predict(training_data)

Generating tree 0 of 50..
Generating tree 5 of 50..
Generating tree 10 of 50..
Generating tree 15 of 50..
Generating tree 20 of 50..
Generating tree 25 of 50..
Generating tree 30 of 50..
Generating tree 35 of 50..
Generating tree 40 of 50..
Generating tree 45 of 50..


In [498]:
print(np.sum(random_training_prediction == training_labels) / training_labels.shape[0])
random_test_prediction = random_forest_classifier.predict(test_data)

0.835869698081


In [497]:
with open("random_forest_census.csv", "w") as f:
    f.write("Id,Category\n")
    for i, val in enumerate(random_test_prediction):
        f.write("{},{}\n".format(i+1, val))
    f.close()

In [519]:
random_data_point = [training_data[1234]]
tree_classifier.predict(random_data_point)

relationship 0
[1 2 3 4] [0 5]
education-num 5
11
capital-gain 0
5013
occupation 6
[ 1  2  4 10 12 13] [ 3  5  6  7  8  9 11 14]
education 6
[ 0  1  2  3  4  5  6 13] [ 8 11 15]
age 39
66
age 39
37
education-num 5
3
fnlwgt 347434
36209
capital-loss 0
1740
fnlwgt 347434
64102
native-country 26
[12  6 14 38 28  3  7 17 27 11  8 31 13 32  1 35] [ 2  4  5 21 22 26 30 33 36 39 41]
workclass 4
[5] [1 2 3 4 6 7]
hours-per-week 43
45


array([0])

In [534]:
from collections import defaultdict
cnt = defaultdict(lambda: 0)
d = defaultdict(lambda: 0)
for root in random_forest_classifier.roots:
    if root.is_categorical_node:
        d[features[root.split_feature]] = [root.left_subset_vals, root.right_subset_vals]
        cnt[features[root.split_feature]] += 1
    else:
        d[features[root.split_feature]] = root.split_threshold
        cnt[features[root.split_feature]] += 1
for k, v in d.items():
    print(k, v, cnt[k])

education-num 12 6
capital-gain 6849 7
relationship [array([1, 2, 3, 4]), array([0, 5])] 15
hours-per-week 40 1
marital-status [array([1, 2]), array([0, 3, 4, 5, 6])] 9
occupation [array([ 1,  3,  5,  6,  7,  8, 13]), array([ 2,  4,  9, 10, 11, 12, 14])] 6
age 28 5
sex [array([0]), array([1])] 1


In [ ]:
workclass ['?' 'Federal-gov' 'Local-gov' 'Never-worked' 'Private' 'Self-emp-inc'
 'Self-emp-not-inc' 'State-gov' 'Without-pay']
education ['10th' '11th' '12th' '1st-4th' '5th-6th' '7th-8th' '9th' 'Assoc-acdm'
 'Assoc-voc' 'Bachelors' 'Doctorate' 'HS-grad' 'Masters' 'Preschool'
 'Prof-school' 'Some-college']
marital-status ['Divorced' 'Married-AF-spouse' 'Married-civ-spouse'
 'Married-spouse-absent' 'Never-married' 'Separated' 'Widowed']
occupation ['?' 'Adm-clerical' 'Armed-Forces' 'Craft-repair' 'Exec-managerial'
 'Farming-fishing' 'Handlers-cleaners' 'Machine-op-inspct' 'Other-service'
 'Priv-house-serv' 'Prof-specialty' 'Protective-serv' 'Sales'
 'Tech-support' 'Transport-moving']
relationship ['Husband' 'Not-in-family' 'Other-relative' 'Own-child' 'Unmarried' 'Wife']
race ['Amer-Indian-Eskimo' 'Asian-Pac-Islander' 'Black' 'Other' 'White']
sex ['Female' 'Male']
native-country ['?' 'Cambodia' 'Canada' 'China' 'Columbia' 'Cuba' 'Dominican-Republic'
 'Ecuador' 'El-Salvador' 'England' 'France' 'Germany' 'Greece' 'Guatemala'
 'Haiti' 'Holand-Netherlands' 'Honduras' 'Hong' 'Hungary' 'India' 'Iran'
 'Ireland' 'Italy' 'Jamaica' 'Japan' 'Laos' 'Mexico' 'Nicaragua'
 'Outlying-US(Guam-USVI-etc)' 'Peru' 'Philippines' 'Poland' 'Portugal'
 'Puerto-Rico' 'Scotland' 'South' 'Taiwan' 'Thailand' 'Trinadad&Tobago'
 'United-States' 'Vietnam' 'Yugoslavia']